In [0]:
!nvidia-smi

In [0]:
!pip install bert-for-tf2

In [3]:
!pip install tensorflow==2.0.0

ERROR: Operation cancelled by user


In [0]:
!pip install sentencepiece

In [0]:
!pip install ktrain

In [7]:
!pip install tensorflow>2.0.0

ERROR: -ensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: -ensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.


In [0]:
from bert import BertModelLayer

In [0]:
import bert
import os 

def createTokenizer():
    currentDir = '/content'
    modelsFolder = '/content'
    vocab_file = os.path.join(modelsFolder, "vocab.txt")

    tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case=False)
    return tokenizer

tokenizer = createTokenizer()

In [0]:
import bert
import os

def createBertLayer():
    global bert_layer

    bertDir = '/content'

    bert_params = bert.params_from_pretrained_ckpt(bertDir)

    bert_layer = bert.BertModelLayer.from_params(bert_params, name="bert")

    bert_layer.apply_adapter_freeze()

def loadBertCheckpoint():
    modelsFolder = '/content'
    checkpointName = os.path.join(modelsFolder, "bert_model.ckpt")

    bert.load_stock_weights(bert_layer, checkpointName)

In [0]:
createBertLayer()

In [14]:
bert_layer

In [0]:
import tensorflow as tf

def createModel():
    global model

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(max_seq_length,), dtype='int32', name='input_ids'),
        bert_layer,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(30, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(30, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(12, activation=tf.nn.softmax)
    ])

    model.build(input_shape=(None, max_seq_length))

    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.05), metrics=['accuracy'])

    print(model.summary())


def fitModel(training_set, training_label, testing_set, testing_label):
    checkpointName = os.path.join('/content', "bert_faq.ckpt")

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointName,
                                                     save_weights_only=True,
                                                     verbose=1)

    # callback = StopTrainingClassComplete()

    history = model.fit(
        training_set,
        training_label,
        epochs=10,
        validation_data=(testing_set, testing_label),
        verbose=1,
        callbacks=[cp_callback]
    )

In [16]:
max_seq_length = 60
createModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (BertModelLayer)        (None, 60, 1024)          332527616 
_________________________________________________________________
flatten (Flatten)            (None, 61440)             0         
_________________________________________________________________
dense (Dense)                (None, 30)                1843230   
_________________________________________________________________
dropout (Dropout)            (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3

In [0]:
import numpy as np

x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [79]:
for l in model.layers:
    print(l.name, l.trainable)

bert False
flatten_3 False
dense_8 True
dropout_79 True
dense_9 True
dropout_80 True
dense_10 True


In [0]:
model.layers[0].trainable= False
    #layer.trainable = False

model.layers[1].trainable = False


In [81]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (BertModelLayer)        (None, 60, 1024)          332527616 
_________________________________________________________________
flatten_3 (Flatten)          (None, 61440)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 30)                1843230   
_________________________________________________________________
dropout_79 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_80 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 12)               

In [82]:
fitModel(x_train[0], y_train, x_test[0], y_test)

Train on 421 samples, validate on 208 samples
Epoch 1/10


ResourceExhaustedError: ignored

In [16]:
import os
os.getcwd()

'/content'